In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load your dataset (replace 'your_dataset.csv' with your actual file)
dataset = pd.read_csv('your_dataset.csv')

# Split the dataset into training and testing sets
train_size = int(len(dataset) * 0.8)
train, test = dataset[:train_size], dataset[train_size:]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create training data
time_steps = 60  # Adjust as needed
x_train, y_train = [], []
for i in range(time_steps, len(train)):
    x_train.append(scaled_data[i-time_steps:i, 0])
    y_train.append(scaled_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Create testing data
inputs = dataset[len(dataset) - len(test) - time_steps:].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(time_steps, inputs.shape[0]):
    X_test.append(inputs[i-time_steps:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Create LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Make predictions
train_pred = model.predict(x_train)
test_pred = model.predict(X_test)

# Inverse transform the predictions
train_pred = scaler.inverse_transform(train_pred)
test_pred = scaler.inverse_transform(test_pred)

# Calculate RMSE
train_rmse = np.sqrt(np.mean((train[:, 0] - train_pred[:, 0])**2))
test_rmse = np.sqrt(np.mean((test[:, 0] - test_pred[:, 0])**2))

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)